In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import cPickle as pickle

In [2]:
### Use a previoulsy trained model and the precomputed extra features to make a prediction on a new test_set

In [3]:





#model_v5_fold0_40.pic
#model_v5_fold2.pic
#model_v6FSter_fold2.pic



In [4]:
file_name = './Final_models/model_v5_fold2.pic'
save_Y_name = './Final_results/Y_priv_v5_fold2.predict'

test_set = pd.read_csv('../data/X_priv.csv')
augmented_features_antoine_test = pd.read_csv('./AugmentedFeaturesPriv.zip')
augmented_features_alex_test = pd.read_csv('./lda_features_5_priv_topics_df.csv')

In [5]:
def try_to_hash(dico, v) :
    try :
        return dico[v]
    except :
        return -1

def preprocess_test_data( df,continuous_factors, OHE_factors, Hashing_dict, Hash_factors,  training_factors ) :
    
    final_factors_test = []
    res_data = pd.DataFrame()
    print 'Creating Dummies'
    for factor in OHE_factors :
        #df[factor].fillna('null', inplace = True)
        print factor
        res_data= pd.concat([res_data, pd.get_dummies(df[factor], prefix = factor)], axis = 1)
        final_factors_test.extend([str(factor)+'_'+str(x) for x in list(set(df[factor]))])
    print 'Done for Dummies!\n'
    
    print 'Hashing Factors...'
    for f in Hash_factors:
        print f
        hashing_dict = Hashing_dict[f]
        res_data[f] = df[f].map(lambda x : try_to_hash(hashing_dict,x))
        final_factors_test.append(f)
    print 'Done for HashFactors!\n'
    
    print 'Continuous Factors'
    final_factors_test.extend(continuous_factors)
    res_data =pd.concat([res_data,df[continuous_factors] ], axis = 1)
    
    print '---------------'
    print 'Missing Factors'
    for f in training_factors :
        if f not in final_factors_test :
            print f
            res_data[f] = -1
    #res_data.fillna(-1, inplace = True)      
    res_data = res_data[training_factors]
    return (np.matrix(res_data.values), final_factors_test)

In [6]:


saved_results = pickle.load(open(file_name, 'rb'))

model, continuous_var, OHE_factors, Hashing_dict, Hash_factors, final_factors  = saved_results['model'], saved_results['continuous_factors'], saved_results['OHE_factors'], saved_results['Hashing_dict'], saved_results['Hash_factors'], saved_results['training_factors']

In [7]:
## download test set and verify nul values!

In [8]:


test_set['IsPolicyholderAHomeowner'] = test_set['IsPolicyholderAHomeowner'].fillna(-1)
test_set['SCID'] = test_set['SCID'].fillna(-1)


to_0_1 = ['CarFuelId', 'CarTransmissionId']
test_set[to_0_1] = test_set[to_0_1].apply(lambda x : x - 1)

In [9]:


feat_name = [ u'VariationCarUsageId',
       u'VariationFirstDriverDrivingLicenseNumberY',
       u'VariationCoverIsNoClaimDiscountSelected',
       u'VariationIsPolicyholderAHomeowner', u'isMostFrequentCarMakeId',
       u'VariationVoluntaryExcess',
       u'VariationPolicyHolderNoClaimDiscountYears', u'VariationCarAge',
       u'VariationCarMakeId', u'rowNumberForUser', u'VariationCarInsuredValue',
       u'VariationCarParkingTypeId',
       u'isMostFrequentFirstDriverDrivingLicenseNumberY',
       u'VariationCarAnnualMileage', u'driverPerCar',
       u'VariationDaysSinceCarPurchase', u'VariationCarDrivingEntitlement']

#continuous_var = continuous_var + feat_name

test_set = pd.concat([test_set, augmented_features_antoine_test[feat_name]], axis = 1)

In [10]:
## Features Alex

In [11]:

feature_name_alex = augmented_features_alex_test.columns.tolist()


test_set = pd.concat([test_set, augmented_features_alex_test[feature_name_alex]], axis = 1)

In [12]:
def simplify_scid(x) :
    if x == 'B01851' :
        return 0
    elif x == 'A04402' :
        return 1
    elif x == 'A10099' :
        return 2
    elif x == 'B02196' :
        return 3
    elif x == 'A08213' :
        return 4
    elif x == 'A03440' :
        return 5
    elif x == 'A04955' :
        return 6
    elif x == 'A09963' :
        return 7
    elif x == 'B01604' :
        return 8
    elif x == 'A06439' :
        return 9
    else :
        return 10
    
def simplify_affinity(x) :
    
    if x == 0 :
        return 0
    elif x == 31 :
        return 1
    elif x==39 :
        return 2
    else :
        return 3
    
def simplify_DrivingLicense(x) :
    if (x % 10) in [0,1] :
        return 0
    elif (x % 10) in [5,6] :
        return 1
    else :
        return 2

In [13]:
test_set.SCID = test_set.SCID.apply(simplify_scid)
test_set.AffinityCodeId = test_set.AffinityCodeId.apply(simplify_affinity)
test_set.FirstDriverDrivingLicenseNumberY = test_set.FirstDriverDrivingLicenseNumberY.apply(simplify_DrivingLicense)
print 'Done!'

Done!


In [14]:
X_test, final_factors_test = preprocess_test_data(test_set, continuous_var, OHE_factors, Hashing_dict, Hash_factors, final_factors  )


Creating Dummies
SCID
FirstDriverDrivingLicenseNumberY
FirstDriverMaritalStatus
CarUsageId
CarParkingTypeId
FirstDriverDrivingLicenceType
CarDrivingEntitlement
CarMakeId
NameOfPolicyProduct
AffinityCodeId
Done for Dummies!

Hashing Factors...
Done for HashFactors!

Continuous Factors
---------------
Missing Factors
CarMakeId_37.0


In [15]:
X_test.shape

(302047, 137)

In [16]:
dtest = xgb.DMatrix(X_test)
dtest = xgb.DMatrix(X_test)
predictions_test = model.predict(dtest, ntree_limit=model.best_ntree_limit)

In [17]:
def create_submission_file(predictions_test) :
    df = pd.DataFrame([ {'Converted': e} for e in predictions_test])
    df.to_csv(save_Y_name, index = None, header = None)

In [18]:
create_submission_file(predictions_test)